# Music Recommendation on Spotify Data

This example demonstrates how you can use KDB.AI to perform similarity recommendations using vector embeddings created from both categorical and numeric music data.

Applications like Spotify and YouTube Music perform hundreds of millions of song recommendations for users every single day. They do this by extracting a vast array of features about every given song and artist and comparing their characteristics.
By leveraging this sort of data, KDB.AI can be used to productionize a music recommendation system and help to quickly and efficiently find music similar to given input songs.

### Aim

In this tutorial, we'll break down how you might perform similarity search on music, taking some Spotify data as an example and using KDB.AI as the vector database to store and query this data.
This breaks down as follows:

1. Load Song Data
1. Create Song Vector Embeddings
1. Store Embeddings In KDB.AI
1. Search For Similar Songs To A Target Song

---

## 0. Setup

### Import Packages

We will start by importing all of the Python packages needed to run this music recommendation system example.
This includes packages for reading in the data, embedding it as vectors, and interacting with the vector database.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# embedding categorical data
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec

In [3]:
# embedding numeric data
from sklearn.preprocessing import StandardScaler

In [4]:
# vector DB
import os
import kdbai_client as kdbai
from getpass import getpass
import time

### Configure Console

In order to fully view our embeddings when it comes to displaying the results, we must increase the maximum allowed column width in Pandas DataFrames from the default value.

In [5]:
pd.set_option("max_colwidth", 1000)

This removes a warning that appears when performing in-place column assignment.

In [6]:
pd.options.mode.chained_assignment = None

### Define Helper Functions

Defining these two helper functions will allow us to easily show the shape and head of any Pandas DataFrames or embedding arrays passed.

In [7]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

In [8]:
def show_embeddings(embeddings: np.array) -> list[int]:
    print("Num Embeddings:", len(embeddings))
    print("Embedding Size:", len(embeddings[0]))
    return list(embeddings[0])

## 1. Load Song Data

The song data we will read in will be taken from an [open-source Spotify dataset](https://www.kaggle.com/datasets/vatsalmavani/spotify-dataset) on Kaggle. There are 5 files on Kaggle, however, only one file is relevant to this analysis.
This dataset contains a list of metadata on 170,000 songs from 1921 to 2020. This metadata includes:
- Song Name
- Artist Name
- Song Year
- Various features about the song's music, including:
    * acousticness
    * danceability
    * duration_ms
    * energy
    * explicit
    * instrumentalness
    * key
    * liveness
    * loudness
    * mode
    * popularity
    * release_date
    * speechiness
    * tempo
    * valence
   

### Read In The Spotify Data From The CSV

We can read this song data from a CSV into a Pandas DataFrame and show the resulting table.

In [9]:
raw_song_df = pd.read_csv("data/song_data.csv")

In [10]:
show_df(raw_song_df)

(170653, 19)


,id,name,artists,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,release_date,speechiness,tempo,valence,year
0,4BJqT0PrAfrxzMOxytFOIz,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,1921,0.0366,80.954,0.0594,1921
1,7xPhfUan2yNtyFG0cUWkt8,Clancy Lowered the Boom,['Dennis Day'],0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,1921,0.4150,60.936,0.9630,1921
2,1o6I8BglA6ylDMrIELygv1,Gati Bali,['KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat'],0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,1921,0.0339,110.339,0.0394,1921
3,3ftBPsC5vPBKxYSee08FDH,Danny Boy,['Frank Parker'],0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,1921,0.0354,100.109,0.1650,1921
4,4d6HGyGT8e121BsdKmw9v6,When Irish Eyes Are Smiling,['Phil Regan'],0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,1921,0.0380,101.665,0.2530,1921


### Pre-process The Data

Here we will perform a few operations on this Pandas DataFrame to get it into the correct format for creating the vector embeddings for our vector database.
This will include:
- Adding a column prefix
- Removing excess columns
- Fixing column values
- Combining columns into one
- Removing duplicate rows

Once these pre-processing steps have been carried out, our data will be clean and in the correct format to start creating embeddings.

In [11]:
# add "song_" prefix to col names
song_df = raw_song_df.add_prefix("song_")

In [12]:
# drop unused cols
song_df = song_df.drop(columns=["song_id", "song_release_date"])

In [13]:
# fix artists list names - remove quotes
def fix_artists(str_list):
    return ", ".join([v for v in str_list.rstrip("']").lstrip("['").split("', '")])


song_df["song_artists"] = song_df["song_artists"].apply(fix_artists)

In [14]:
# combine song_name & song_artists into song_description
song_df.insert(
    0, "song_description", song_df["song_name"] + " - " + song_df["song_artists"]
)

In [15]:
# remove duplicate rows
song_data = song_df[
    ~song_df.duplicated(subset=["song_description"], keep="first")
].reset_index(drop=True)

In [16]:
show_df(song_df)

(170653, 18)


,song_description,song_name,song_artists,song_acousticness,song_danceability,song_duration_ms,song_energy,song_explicit,song_instrumentalness,song_key,song_liveness,song_loudness,song_mode,song_popularity,song_speechiness,song_tempo,song_valence,song_year
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve - Sergei Rachmaninoff, James Levine, Berliner Philharmoniker","Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,0.0366,80.954,0.0594,1921
1,Clancy Lowered the Boom - Dennis Day,Clancy Lowered the Boom,Dennis Day,0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,0.4150,60.936,0.9630,1921
2,Gati Bali - KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,0.0339,110.339,0.0394,1921
3,Danny Boy - Frank Parker,Danny Boy,Frank Parker,0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,0.0354,100.109,0.1650,1921
4,When Irish Eyes Are Smiling - Phil Regan,When Irish Eyes Are Smiling,Phil Regan,0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,0.0380,101.665,0.2530,1921


## 2. Create Song Vector Embeddings

We will create vector embeddings from this data in three steps:

- A. Encoding the categorical `song_description` column as numeric values
- B. Scaling the numeric column values
- C. Joining these two sets of encodings together into one vector embedding

### A. Embed Categorical Song Metadata

To embed the `song_description` column as numeric vectors, we must perform natural language processing on them.
This involves tokenising the descriptions to break them up into their individual sub-parts and then using a `Word2Vec` model to turn these tokenised song descriptions into vectors.

The length of the vectors we turn these desciptions into is configurable, however, in this case we chose to set this to `15` as there are also 15 numeric columns which describe the song. We do not want to bias the final embedding vectors in favour of either the categorical columns or the numeric columns, so it made sense to keep the number of values representing each the same for both.

In [17]:
# tokenize the descriptions
tokenised_song_descs = [word_tokenize(v.lower()) for v in song_data["song_description"]]

In [18]:
# create embedding model
embedding_dim = 15

word2Vec_model = Word2Vec(
    sentences=tokenised_song_descs,
    vector_size=embedding_dim,
    window=5,
    min_count=1,
    sg=1,
)

In [19]:
# function to create embedding vector from tokens
def get_embedding(song_desc_tokens, model, embedding_dim):
    vectors = [model.wv[token] for token in song_desc_tokens if token in model.wv]

    # Average of word vectors OR zeros if no valid tokens found
    return sum(vectors) / len(vectors) if vectors else [0] * embedding_dim

In [20]:
# embed song descriptions as vectors
categorical_embeddings = [
    get_embedding(song_desc_tokens, word2Vec_model, embedding_dim)
    for song_desc_tokens in tokenised_song_descs
]

In [21]:
show_embeddings(categorical_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[-1.3061829,
 1.449745,
 1.2838475,
 -0.37391582,
 0.62907904,
 -0.0010437369,
 -1.4169772,
 -0.21957593,
 -0.3732935,
 1.2440405,
 0.8359612,
 0.15343893,
 0.5153092,
 -0.5346563,
 -0.9461686]

### B. Embed Numeric Song Metadata

There are 15 numeric columns in our data which we will use to make up the other half of our final embedding vectors.
First, however, we will scale these values to make them more uniform.

The standard scaled score of a sample `x` is calculated as:

$$
    z = \frac{(x - u)}{s}
$$

where `u` is the mean of the training samples and `s` is the standard deviation of the training samples.

In [22]:
# extract numeric columns
numeric_cols = list(
    song_data.drop(columns=["song_name", "song_artists", "song_description"]).columns
)
numeric_cols

['song_acousticness',
 'song_danceability',
 'song_duration_ms',
 'song_energy',
 'song_explicit',
 'song_instrumentalness',
 'song_key',
 'song_liveness',
 'song_loudness',
 'song_mode',
 'song_popularity',
 'song_speechiness',
 'song_tempo',
 'song_valence',
 'song_year']

In [23]:
# scale these columns
numeric_embeddings = StandardScaler().fit_transform(song_data[numeric_cols])

In [24]:
show_embeddings(numeric_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[1.2703070294949106,
 -1.461259048884883,
 4.752569009266444,
 -1.007676175100162,
 -0.3092011481361043,
 2.262496351074803,
 1.3649563314116429,
 2.6110012104955738,
 -1.5078176079821606,
 0.6453499264358126,
 -1.2499471942272533,
 -0.38364744367670833,
 -1.1655450558051375,
 -1.7786347004763523,
 -2.142666230649]

### C. Merge Categorical & Numeric Embeddings

This leaves us with two sets of vectors: one representing the categorical column and one representing the numeric columns.
Both sets have 15 values each, so when we join these together, the resulting vector will have 30 values.

In [25]:
row_embeddings = [
    np.concatenate([cat_row, num_row])
    for cat_row, num_row in zip(categorical_embeddings, numeric_embeddings)
]

In [26]:
show_embeddings(row_embeddings)

Num Embeddings: 157685
Embedding Size: 30


[-1.306182861328125,
 1.4497450590133667,
 1.283847451210022,
 -0.37391582131385803,
 0.6290790438652039,
 -0.0010437369346618652,
 -1.4169771671295166,
 -0.2195759266614914,
 -0.3732934892177582,
 1.2440404891967773,
 0.8359612226486206,
 0.15343892574310303,
 0.51530921459198,
 -0.534656286239624,
 -0.9461686015129089,
 1.2703070294949106,
 -1.461259048884883,
 4.752569009266444,
 -1.007676175100162,
 -0.3092011481361043,
 2.262496351074803,
 1.3649563314116429,
 2.6110012104955738,
 -1.5078176079821606,
 0.6453499264358126,
 -1.2499471942272533,
 -0.38364744367670833,
 -1.1655450558051375,
 -1.7786347004763523,
 -2.142666230649]

### Create DataFrame With Embeddings

We can take these defined embeddings and create a Pandas DataFrame containing them.
This will be the table we insert into our vector database.

To enable proper filtering of the data once inserted into the KDB.AI vector database, we will pair these embedding vectors with three song description columns: `song_name`, `song_artists`, and `song_year`.

In [27]:
embedded_song_df = song_data[["song_name", "song_artists", "song_year"]]

In [28]:
embedded_song_df["song_embeddings"] = row_embeddings

In [29]:
show_df(embedded_song_df)

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.306182861328125, 1.4497450590133667, 1.283847451210022, -0.37391582131385803, 0.6290790438652039, -0.0010437369346618652, -1.4169771671295166, -0.2195759266614914, -0.3732934892177582, 1.2440404891967773, 0.8359612226486206, 0.15343892574310303, 0.51530921459198, -0.534656286239624, -0.9461686015129089, 1.2703070294949106, -1.461259048884883, 4.752569009266444, -1.007676175100162, -0.3092011481361043, 2.262496351074803, 1.3649563314116429, 2.6110012104955738, -1.5078176079821606, 0.6453499264358126, -1.2499471942272533, -0.38364744367670833, -1.1655450558051375, -1.7786347004763523, -2.142666230649]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.29974231123924255, 0.761445164680481, 1.224726676940918, 0.10159605741500854, 0.7863611578941345, 0.7182739973068237, -0.724828839302063, 0.01951512321829796, -0.19665169715881348, -0.04318639636039734, 0.8203275799751282, 0.3068835437297821, 0.6379690766334534, -0.533280074596405, 0.09706346690654755, 0.6055353575765545, 1.600008527700499, -0.3958314054754836, -0.5219481676317469, -0.3092011481361043, -0.5349944602375606, 0.5117210944216953, -0.26640400859153673, -0.1646257835500665, 0.6453499264358126, -1.2044048830284118, 1.872793548131166, -1.8164526201212032, 1.6541851866630062, -2.142666230649]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.34657829999923706, 0.5139216184616089, -0.23727956414222717, 0.5176736116409302, 1.0185598134994507, 0.16897541284561157, 0.13408352434635162, -0.06599444150924683, -0.24015653133392334, 0.28537827730178833, 0.7764918208122253, 0.19054535031318665, 0.36814096570014954, 0.4701331853866577, -0.9946604371070862, 1.2144662090537686, -1.183477361379913, 2.1306274127125904, -1.1758127930699978, -0.3092011481361043, 2.374013638541697, -0.6259258882315683, -0.6025761034947833, -0.5873232499193564, 0.6453499264358126, -1.2044048830284118, -0.3997478418740689, -0.21005905433508043, -1.854615663007104, -2.142666230649]"
3,Danny Boy,Frank Parker,1921,"[-0.5785746574401855, 1.0987048149108887, 1.7012841701507568, 0.0821155458688736, 0.6395443081855774, 0.5136593580245972, -0.8803048133850098, 0.221713587641716, -0.030609577894210815, 0.20444893836975098, 1.0348442792892456, 0.35724061727523804, 1.0183840990066528, -0.5400786995887756, 0.3654381334781647, 1.2304207291798093, -1.4839351050077376, -0.16284109162119192, -0.6415119848547415, -0.3092011481361043, -0.5349062022700511, -0.05710239690493648, 0.9928168892663868, 0.3837053008849818, 0.6453499264358126, -1.295489505426095, -0.39080317620886856, -0.5426988976237884, -1.3774552183139837, -2.142666230649]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.27261143922805786, 0.833780825138092, 0.8910992741584778, -0.013010159134864807, 1.0156773328781128, 0.7111427783966064, -1.197956919670105, 0.241031214594841, -0.11696281284093857, -0.20735326409339905, 1.3338439464569092, -0.01072670891880989, 0.6512774229049683, -0.44728603959083557, 0.10168053209781647, 1.203829862303075, -0.6732660986156829, -0.5052618172494476, -1.0749308222880962, -0.3092011481361043, -0.5349891074077622, -0.6259258882315683, 0.12674640748175162, 0.24684186220999385, 0.6453499264358126, -1.3410318166249366, -0.3752990890558546, -0.4921038246856425, -1.0431389831786766, -2.142666230649]"


## 3. Store Embeddings In KDB.AI

Now that our data has been read in and our embeddings have been defined, we can set about creating a KDB.AI vector database and ingesting the data into it.

Once we have the database set up, we can then query our data and start deriving some insights

### Connect to KDB.AI Session

To use KDB.AI, you will need two session details - a URL endpoint and an API key. To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [30]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [31]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

We can check if there are any tables defined in this KDB.AI session - it should not contain a table called `songs`.

In [32]:
session.list()

[]

### Define Vector DB Table Schema

The next step is to define the schema for the table in KDB.AI which will store our embeddings.

As mentioned above, our table will have four columns:
- Song Name
- Song Artists
- Song Year
- Song Embeddings

When defining the schema, we must supply the types of these columns. We can use the `.dtypes()` function on the defined Pandas DataFrame to help with this.

In [33]:
embedded_song_df.dtypes

song_name          object
song_artists       object
song_year           int64
song_embeddings    object
dtype: object

In [34]:
schema = {
    "columns": [
        {
            "name": "song_name",
            "pytype": "str",
        },
        {
            "name": "song_artists",
            "pytype": "str",
        },
        {
            "name": "song_year",
            "pytype": "int64",
        },
        {
            "name": "song_embeddings",
            "vectorIndex": {
                "dims": len(numeric_cols) + embedding_dim,
                "metric": "L2",
                "type": "flat",
            },
        },
    ]
}

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [35]:
# First ensure the table does not already exist
try:
    session.table("songs").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [36]:
table = session.create_table("songs", schema)

### Add Embedded Data to KDB.AI Table

When adding larger amounts of data, you should insert data into an index in chunks.

It is a good idea to first get an idea of how large your dataset to insert is.

In [37]:
embedded_song_df.memory_usage(deep=True).sum() / (1024**2)

79.63715362548828

This dataset is 80MB which exceeds the insert limit of <10MB at a time. As such, we'll insert this data in chunks, inserting 10,000 rows at a time.

In [38]:
chunk_size = 10_000

In [39]:
for i in range((len(embedded_song_df) // chunk_size) + 1):
    index = i * chunk_size
    table.insert(
        embedded_song_df.iloc[index : index + chunk_size].reset_index(drop=True)
    )

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [40]:
show_df(table.query())

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.306182861328125, 1.4497450590133667, 1.283847451210022, -0.37391582131385803, 0.6290790438652039, -0.0010437369346618652, -1.4169771671295166, -0.2195759266614914, -0.3732934892177582, 1.2440404891967773, 0.8359612226486206, 0.15343892574310303, 0.51530921459198, -0.534656286239624, -0.9461686015129089, 1.2703070294949106, -1.461259048884883, 4.752569009266444, -1.007676175100162, -0.3092011481361043, 2.262496351074803, 1.3649563314116429, 2.6110012104955738, -1.5078176079821606, 0.6453499264358126, -1.2499471942272533, -0.38364744367670833, -1.1655450558051375, -1.7786347004763523, -2.142666230649]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.29974231123924255, 0.761445164680481, 1.224726676940918, 0.10159605741500854, 0.7863611578941345, 0.7182739973068237, -0.724828839302063, 0.01951512321829796, -0.19665169715881348, -0.04318639636039734, 0.8203275799751282, 0.3068835437297821, 0.6379690766334534, -0.533280074596405, 0.09706346690654755, 0.6055353575765545, 1.600008527700499, -0.3958314054754836, -0.5219481676317469, -0.3092011481361043, -0.5349944602375606, 0.5117210944216953, -0.26640400859153673, -0.1646257835500665, 0.6453499264358126, -1.2044048830284118, 1.872793548131166, -1.8164526201212032, 1.6541851866630062, -2.142666230649]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.34657829999923706, 0.5139216184616089, -0.23727956414222717, 0.5176736116409302, 1.0185598134994507, 0.16897541284561157, 0.13408352434635162, -0.06599444150924683, -0.24015653133392334, 0.28537827730178833, 0.7764918208122253, 0.19054535031318665, 0.36814096570014954, 0.4701331853866577, -0.9946604371070862, 1.2144662090537686, -1.183477361379913, 2.1306274127125904, -1.1758127930699978, -0.3092011481361043, 2.374013638541697, -0.6259258882315683, -0.6025761034947833, -0.5873232499193564, 0.6453499264358126, -1.2044048830284118, -0.3997478418740689, -0.21005905433508043, -1.854615663007104, -2.142666230649]"
3,Danny Boy,Frank Parker,1921,"[-0.5785746574401855, 1.0987048149108887, 1.7012841701507568, 0.0821155458688736, 0.6395443081855774, 0.5136593580245972, -0.8803048133850098, 0.221713587641716, -0.030609577894210815, 0.20444893836975098, 1.0348442792892456, 0.35724061727523804, 1.0183840990066528, -0.5400786995887756, 0.3654381334781647, 1.2304207291798093, -1.4839351050077376, -0.16284109162119192, -0.6415119848547415, -0.3092011481361043, -0.5349062022700511, -0.05710239690493648, 0.9928168892663868, 0.3837053008849818, 0.6453499264358126, -1.295489505426095, -0.39080317620886856, -0.5426988976237884, -1.3774552183139837, -2.142666230649]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.27261143922805786, 0.833780825138092, 0.8910992741584778, -0.013010159134864807, 1.0156773328781128, 0.7111427783966064, -1.197956919670105, 0.241031214594841, -0.11696281284093857, -0.20735326409339905, 1.3338439464569092, -0.01072670891880989, 0.6512774229049683, -0.44728603959083557, 0.10168053209781647, 1.203829862303075, -0.6732660986156829, -0.5052618172494476, -1.0749308222880962, -0.3092011481361043, -0.5349891074077622, -0.6259258882315683, 0.12674640748175162, 0.24684186220999385, 0.6453499264358126, -1.3410318166249366, -0.3752990890558546, -0.4921038246856425, -1.0431389831786766, -2.142666230649]"


## 4. Search For Similar Songs To A Target Song

Now that the data has been inserted into the database, we can perform some queries on the data.

### Find Songs By A Certain Artist

We can query the database to find songs by particular artists using KDB.AI's `.query()` function.

Here, we want to return all songs in the dataset by the DJ `Calvin Harris`, sorted by the year they were produced. This returns 32 songs to us.

In [41]:
table.query(filter=[("like", "song_artists", "*Calvin Harris*")], sort_by="song_year")

,song_name,song_artists,song_year,song_embeddings
0,Flashback,Calvin Harris,2009,"[-0.5308306217193604, 0.8614987730979919, 1.1055147647857666, -0.08214779198169708, 0.9314597845077515, 0.48097139596939087, -0.819793701171875, -0.2747561037540436, 0.33063453435897827, -0.007644712924957275, 0.7431244850158691, 0.5099509954452515, 0.7296927571296692, 0.18614216148853302, 0.3438257575035095, -1.3351724705550174, -1.5406252453148743, -0.011030115749796763, 1.7460279903169285, -0.3092011481361043, -0.5313940449564866, 1.0805445857483271, -0.7039975151774578, 1.0250333372402145, -1.549546934208044, 0.7539144985217788, -0.307319630000332, 0.36420802995835755, -1.2520866301382436, 1.247808968405853]"
1,You Used To Hold Me,Calvin Harris,2009,"[-0.015125058591365814, 1.257551670074463, 1.3064601421356201, -0.15788251161575317, 0.9448271989822388, 0.6127932071685791, -1.2258079051971436, 0.2828027307987213, 0.2980310022830963, -0.010570086538791656, 1.607230544090271, 0.05081271007657051, 0.8250176906585693, 0.042660534381866455, 0.17760735750198364, -1.279491195315136, 0.46053670752705134, 0.004253770518922344, 1.8244917453695186, -0.3092011481361043, -0.5349904137531296, 1.6493680770749588, 0.6224578016611152, 0.958707209266951, -1.549546934208044, 0.5717452537264123, -0.35562082459241384, 0.3955535635781341, -1.2824790151505443, 1.247808968405853]"
2,I'm Not Alone - Radio Edit,Calvin Harris,2009,"[-0.39972418546676636, 1.1283369064331055, 1.1839702129364014, -0.13868026435375214, 0.7734965085983276, 0.7137905359268188, -1.1721179485321045, 0.27151015400886536, 0.23957271873950958, 0.1312033236026764, 1.3197147846221924, 0.21488842368125916, 0.7364214658737183, -0.131475567817688, 0.24552437663078308, -1.3273547556932577, 0.3074733286977821, -0.1491385696844665, 0.7932538218211915, -0.3092011481361043, -0.14309084999676247, 0.5117210944216953, 0.4914076629700189, 0.8665875870818629, 0.6453499264358126, 0.3895760089310457, -0.411077751716656, 0.46191895166730823, -0.36310936852844955, 1.247808968405853]"
3,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3097107410430908, 1.2280445098876953, 1.2125667333602905, -0.1728026419878006, 0.8294687271118164, 0.48141586780548096, -1.1926295757293701, 0.1443655639886856, 0.34478357434272766, 0.09496279805898666, 1.2233936786651611, 0.3410244882106781, 0.9546244144439697, -0.12350010871887207, 0.1781793236732483, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]"
4,Dance Wiv Me - Radio Edit,"Dizzee Rascal, Calvin Harris, Chrome",2011,"[-0.3572603762149811, 0.9557803273200989, 1.0952162742614746, -0.06817266345024109, 0.7185381650924683, 0.43593087792396545, -0.6681462526321411, 0.10622109472751617, 0.051009584218263626, 0.12988132238388062, 0.751168429851532, 0.2921321988105774, 0.5334352850914001, -0.01799139566719532, 0.14057756960391998, -1.2143435714671371, 1.9344803555126058, -0.20954668716662583, 0.9912813940967762, -0.3092011481361043, -0.5349944602375606, 1.6493680770749588, -0.30059100129356187, 1.2671763441267319, 0.6453499264358126, 1.5281337889020867, -0.3329610049072396, -0.1561798539118754, 1.00454795702508, 1.3248652229298272]"
5,Feel So Close - Radio Edit,Calvin Harris,2012,"[-0.42053648829460144, 1.1180672645568848, 1.255746841430664, -0.21703937649726868, 0.8270436525344849, 0.7070026993751526, -1.0651367902755737, 0.12035082280635834, 0.007855415344238281, -0.002819826127961278, 1.2384101152420044, 0.24876461923122406, 0.8643861413002014, 0.09457863867282867, 0.38695913553237915, -1.3383314655399736, 0.9650789562605678, -0.19120286091549893, 1.6563551273996828, -0.3092011481361043, -0.5125954164977816, 0.5117210944216953, -0.015699395443352974, 1.5196718418873827, 0.6453499264358126, 2.2112684568847114, -0.4170408621601229, 

### Find A Specific Song

We can filter this query further by looking for the song `We Found Love` by `Calvin Harris` in our dataset.

This will only return one song as he only produced one song with this name.

In [42]:
table.query(
    filter=[
        ("like", "song_artists", "*Calvin Harris*"),
        ("like", "song_name", "*We Found Love*"),
    ]
)

,song_name,song_artists,song_year,song_embeddings
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3097107410430908, 1.2280445098876953, 1.2125667333602905, -0.1728026419878006, 0.8294687271118164, 0.48141586780548096, -1.1926295757293701, 0.1443655639886856, 0.34478357434272766, 0.09496279805898666, 1.2233936786651611, 0.3410244882106781, 0.9546244144439697, -0.12350010871887207, 0.1781793236732483, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]"


### Find Similar Songs To This Song

We can then copy and paste the vector associated with this song below and save it as the variable `my_vec`.

We will then use KDB.AI's `.search()` function to find similar songs in the dataset to this song using this vector.
We will pull out the 5 songs most similar to this song from the dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> The most similar song will be "We Found Love" by "Calvin Harris" as this is the vector we are using for the search.
</div>

In [43]:
my_vec = [-0.3097107410430908, 1.2280445098876953, 1.2125667333602905, -0.1728026419878006, 0.8294687271118164, 0.48141586780548096, -1.1926295757293701, 0.1443655639886856, 0.34478357434272766, 0.09496279805898666, 1.2233936786651611, 0.3410244882106781, 0.9546244144439697, -0.12350010871887207, 0.1781793236732483, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]

In [44]:
table.search(vectors=[my_vec], n=5)[0]

,song_name,song_artists,song_year,song_embeddings,__nn_distance
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3097107410430908, 1.2280445098876953, 1.2125667333602905, -0.1728026419878006, 0.8294687271118164, 0.48141586780548096, -1.1926295757293701, 0.1443655639886856, 0.34478357434272766, 0.09496279805898666, 1.2233936786651611, 0.3410244882106781, 0.9546244144439697, -0.12350010871887207, 0.1781793236732483, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]",0.000000
1,Bad At Love,Halsey,2017,"[-0.3356478810310364, 1.0972840785980225, 1.2874135971069336, 0.19179025292396545, 0.6721011400222778, 0.5797968506813049, -1.0475871562957764, 0.47525662183761597, 0.10440023988485336, 0.13717558979988098, 1.3927897214889526, 0.08936907351016998, 0.8013035655021667, -0.47465166449546814, 0.31363052129745483, -1.1803072618649173, 0.7836705072777309, -0.3899329165171471, 1.0099632405378691, -0.3092011481361043, -0.5349944602375606, -1.479161125221516, -0.6692407392637322, 1.3973720768149898, 0.6453499264358126, 1.9380145896916616, -0.4253892167809766, 0.051533080489714826, 0.32071929424831513, 1.556033986501749]",1.111526
2,Sweet but Psycho,Ava Max,2020,"[-0.3523186147212982, 0.9123393893241882, 0.9346564412117004, -0.1309065967798233, 0.7823386788368225, 0.5283251404762268, -1.143739938735962, 0.3669535219669342, -0.02260895073413849, 0.41644683480262756, 0.9376603960990906, 0.45754924416542053, 0.8685446381568909, -0.28198787569999695, 0.21680830419063568, -1.15903456836353, 1.0387761386598457, -0.3412506155567211, 0.841826622568033, -0.3092011481361043, -0.5349944602375606, -1.1947493795582, -0.23221701588951166, 1.1903222593323153, 0.6453499264358126, 2.393437701680078, -0.31984216193161247, 0.5268536307530907, 0.3511116792606158, 1.67161836828771]",1.156976
3,No One Compares To You,Jack & Jack,2019,"[-0.3300929665565491, 0.9315669536590576, 1.1165355443954468, 0.0159626342356205, 0.7132259607315063, 0.3990531861782074, -1.1266071796417236, 0.06196334585547447, 0.14290468394756317, 0.3190250098705292, 1.5081255435943604, 0.2299031913280487, 0.6675042510032654, -0.5276877880096436, 0.15401417016983032, -1.0032120884658673, 0.9253958580455721, -0.37695782217313883, 0.9987541326732133, -0.3092011481361043, -0.5349944602375606, -1.1947493795582, -0.6418911451021122, 0.9578298795318549, 0.6453499264358126, 1.8924722784928198, -0.3430982926611334, -0.15803327239354273, 0.39290120865252925, 1.6330902410257229]",1.247350
4,Stay Gold,BTS,2020,"[-0.2775978147983551, 0.8558111190795898, 1.0510300397872925, 0.06652789562940598, 0.6991321444511414, 0.6517294049263, -1.0060275793075562, 0.2261066883802414, -0.18408510088920593, 0.11624479293823242, 1.1917176246643066, 0.2641656994819641, 0.6267741322517395, -0.38359335064888, 0.27587199211120605, -1.1037255652599227, 1.0557831807519866, 0.1001951445409801, 0.5728080338162957, -0.3092011481361043, -0.5349944602375606, -1.1947493795582, -0.7296377597039766, 1.0681979602069414, 0.6453499264358126, 2.2112684568847114, -0.2786966998716909, 0.39727691760494743, 0.20674785045218758, 1.67161836828771]",1.349034


### Automate This Song Similarity Search Process

We can define a function to automate this process and find songs which are the most similar to any input song.
This will allow us to use the KDB.AI vector database in a more production-like setting to perform similarity search and music recommendation.

In [47]:
def find_similar_songs(
    vectorDB_song_tab,
    song_name: str,
    song_artists: list[str] = None,
    song_year: int = None,
    n_similar: int = 5,
    exact: bool = False,
) -> None:
    # create filter list
    filter_list = [("like", "song_name", f"{song_name}" if exact else f"*{song_name}*")]
    if song_artists:
        if type(song_artists) == str:
            song_artists = list(song_artists)
        for artist in song_artists:
            filter_list.append(("like", "song_artists", f"*{artist}*"))
    if song_year:
        filter_list.append(("like", "song_year", f"{song_year}"))

    # find songs liks this in vector DB
    resulting_song = vectorDB_song_tab.query(filter=filter_list, sort_by="song_year")

    # quality check
    if resulting_song.empty:
        print(
            "Song Not Found! Please double check the values entered or try another song"
        )
        return

    # find vectors associated with these songs
    resulting_vectors = [v.tolist() for v in resulting_song["song_embeddings"]]

    # search for similar songs to selected songs
    similar_songs = vectorDB_song_tab.search(vectors=resulting_vectors, n=n_similar + 1)

    # process similar song table
    for i, similar_df in enumerate(similar_songs):
        name = resulting_song.loc[i, "song_name"]
        artists = resulting_song.loc[i, "song_artists"]
        year = resulting_song.loc[i, "song_year"]
        print(f"Songs Similar To '{name}' By '{artists}' ({year})")
        for j, song in similar_df[1:].iterrows():
            print(
                f"   {j}. {song['song_name']} - {song['song_artists']} ({song['song_year']})"
            )
        print()

##### Songs by multiple artists

Here, we will query this function to search the KDB.AI vector database to look for the song `Let's Go` by two artists - `Calvin Harris` and `Ne-Yo`. 

In [48]:
find_similar_songs(table, song_name="Let's Go", song_artists=["Calvin Harris", "Ne-Yo"])

Songs Similar To 'Let's Go (feat. Ne-Yo)' By 'Calvin Harris, Ne-Yo' (2012)
   1. I Cry - Flo Rida (2012)
   2. Mmm Yeah (feat. Pitbull) - Austin Mahone, Pitbull (2014)
   3. Too Much (feat. Usher) - Marshmello, Imanbek, Usher (2020)
   4. All Around The World - Justin Bieber, Ludacris (2012)
   5. No Money - Galantis (2016)



If you search for these songs on [YouTube](https://www.youtube.com/), you will see that the results returned have quite a similar vibe to the song we searched for showing the similarity search power of KDB.AI.

##### Specify different number of similar songs

We can adjust the number of results returned to us by specifying the `n_similar` parameter.
Here, we will search for the `8` most similar songs to the song `Californiacation` by `Red Hot Chili Peppers`.

In [49]:
find_similar_songs(
    table,
    song_name="Californication",
    song_artists="Red Hot Chili Peppers",
    n_similar=8,
)

Songs Similar To 'Californication' By 'Red Hot Chili Peppers' (1999)
   1. Police Station - Red Hot Chili Peppers (2011)
   2. Charlie - Red Hot Chili Peppers (2006)
   3. Dark Necessities - Red Hot Chili Peppers (2016)
   4. Especially in Michigan - Red Hot Chili Peppers (2006)
   5. Don't Forget Me - Red Hot Chili Peppers (2002)
   6. Cabron - Red Hot Chili Peppers (2002)
   7. Face Down - The Red Jumpsuit Apparatus (2006)
   8. Look Around - Red Hot Chili Peppers (2011)



When we return these similar songs, you will notice that a lot of the similar songs are by the same artist as the original song we searched for - `Red Hot Chili Peppers`.
This makes sense as their music usually has pretty unique features. 

#### All songs with a given name by any artist

The final thing we will perform a similarity search on all songs stored in the KDB.AI vector database with a given song name - `Love Me`.

In [50]:
find_similar_songs(table, song_name="Love Me", exact=True)

Songs Similar To 'Love Me' By 'Elvis Presley' (1956)
   1. Don't - Elvis Presley (1959)
   2. Without Him - Elvis Presley (1967)
   3. Fine And Mellow - Billie Holiday (1957)
   4. Everything I Have Is Yours - 10'' Version - Billie Holiday (1956)
   5. Harbor Lights - Elvis Presley (1959)

Songs Similar To 'Love Me' By 'Buddy Holly' (1958)
   1. A Love That's Worth Having - Willie Hutch (1969)
   2. Midnight Shift - Buddy Holly (1958)
   3. Johnny be good - Radio Version - Jonny Bombastic (1955)
   4. Lonely Weekends - Wanda Jackson (1961)
   5. Oh, What a Good Thing We Had - Johnny Cash, June Carter Cash (1967)

Songs Similar To 'Love Me' By 'Sarah Vaughan' (1958)
   1. Summer Is Gone - Carmen McRae (1956)
   2. Make the World Go Away - Ray Price (1956)
   3. I'll Come Back For More - Dinah Washington (1961)
   4. Stairway To The Stars - Dinah Washington, Chubby Jackson Orchestra (1956)
   5. Last Night When We Were Young - Carmen McRae (1955)

Songs Similar To 'Love Me' By 'Bo Diddle

There are 15 songs in our vector database with `Love Me` as their title and we have returned the most similar songs to each of these.
With that, we have built a recommendation system which is able to recommend music based both on user numerical and categorical song data.

## 6. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [51]:
table.drop()

True